В качестве текста выбрано введение к первому тому "Капитала" К. Маркса. Размер исходного текста превышает 60 000 символов. Написан на русском языке, встречаются слова на латинице. Содержит различные знаки препинания, ссылки в различных скобках, библиотечные шифры, и прочее.

В качестве вспомогательного материала использовал материал из интернета:
https://habr.com/ru/post/517410/

Импортируем необходимые библиотеки.

In [68]:
import string
import nltk
import numpy as np
import pandas as pd

Откроем файл, прочтём его содержимое в переменную text, которая будет содержать текст, закроем файл

In [69]:
file_obj = open("text.txt")
text = ""
text = file_obj.read()
file_obj.close()

len(text)

64334

Сперва пройдём весь текст и избавимся от большинства знаков препинания, цифр, скобок, знаков табуляции и новой строки. Но так как нам требуется разделить текст на предложения, не будем убирать знаки окончания предложения (точки, знак вопроса, восклицательный знак). Также не будем пока переводить все буквы в нижний регистр. Планирую использовать тот факт, что при использовании точки для сокращения слова (т. д., проч. и другое) следующее за ним слово пишется с маленькой буквы. Такая точка не считается принаком завершения предложения.

In [70]:
spec_symbol = string.punctuation
spec_symbol = spec_symbol.replace(".", "")
spec_symbol = spec_symbol.replace("!", "")
spec_symbol = spec_symbol.replace("?", "")
spec_symbol += string.digits
spec_symbol += "\n\t\'\"”“–"
symbols_of_end_sentence = ".!?"

spec_symbol

'"#$%&\'()*+,-/:;<=>@[\\]^_`{|}~0123456789\n\t\'"”“–'

Произведем замену всех спецсимволов на знак пробела (обозначается символом \x20).

In [71]:
for symbol in spec_symbol:
    text = text.replace(symbol, "\x20")
text = text.replace("...", "\x20")

len(text)

64332

Пройдем весь текст от начала и до конца в цикле. Если встретим символ конца строки, проверим регистр ближайшего следующего за ним символа. Введем логическую переменную meeting_symbol, которая принимает значение True если мы недавно встретили знак конца предложения, но еще не "решили его судьбу". Будем заменять "правильные" символы конца строки на символ "*" (их там уже нет, хотя можно бы и более специфический символ).

In [79]:
meeting_symbol = False
symbol_of_end_sentence = None

for i in range(len(text)):

    if text[i] in symbols_of_end_sentence:
        meeting_symbol = True
        symbol_of_end_sentence = text[i]
        continue

    if meeting_symbol == True and text[i] != "\x20":
        if text[i] == text[i].upper():
            text = text.replace(symbol_of_end_sentence, "\x20" + "*", 1)
            meeting_symbol = False
            symbol_of_end_sentence = None
        else:
            text = text.replace(symbol_of_end_sentence, "\x20", 1)
            meeting_symbol = False
            symbol_of_end_sentence = None

#выловим последний знак препинания, т.к. за ним ничего нет
for _ in symbols_of_end_sentence:
    text = text.replace(_, "\x20" + "*", 1)

len(text)

64752

Создадим список из строки для подсчёта количество предложений и слов в них

In [83]:
text_list = text.split()

len(text_list)

8832

Определим количество предложений и слов в них.

In [86]:
number_of_sentence = text_list.count("*")
word_counter = 0
list_of_words_in_sentence = []

for word in text_list:
    if word == "*":
        list_of_words_in_sentence.append(word_counter)
        word_counter = 0
    else:
        word_counter += 1

number_of_sentence, len(list_of_words_in_sentence)

(420, 420)

Вернемся обрано к строковой переменной. Удалим символы конца предложения, выровняем регистр. 

In [95]:
text = text.replace("*", "")
text = text.lower()

len(text)

64332